In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/")

os.chdir('yolov7')


In [ ]:
import sys
sys.path.append('/content/yolov7/')

In [ ]:
import cv2
import torch
import numpy as np
from pathlib import Path
from utils.datasets import letterbox
from utils.general import non_max_suppression, scale_coords
from utils.plots import plot_one_box
import imutils

In [ ]:
import numpy as np
import random
import torch
import torch.nn as nn

from models.common import Conv, DWConv
from utils.google_utils import attempt_download


class CrossConv(nn.Module):
    def __init__(self, c1, c2, k=3, s=1, g=1, e=1.0, shortcut=False):
        super(CrossConv, self).__init__()
        c_ = int(c2 * e)
        self.cv1 = Conv(c1, c_, (1, k), (1, s))
        self.cv2 = Conv(c_, c2, (k, 1), (s, 1), g=g)
        self.add = shortcut and c1 == c2

    def forward(self, x):
        return x + self.cv2(self.cv1(x)) if self.add else self.cv2(self.cv1(x))


class Sum(nn.Module):
    def __init__(self, n, weight=False):
        super(Sum, self).__init__()
        self.weight = weight
        self.iter = range(n - 1)
        if weight:
            self.w = nn.Parameter(-torch.arange(1., n) / 2, requires_grad=True)

    def forward(self, x):
        y = x[0]
        if self.weight:
            w = torch.sigmoid(self.w) * 2
            for i in self.iter:
                y = y + x[i + 1] * w[i]
        else:
            for i in self.iter:
                y = y + x[i + 1]
        return y


class MixConv2d(nn.Module):
    def __init__(self, c1, c2, k=(1, 3), s=1, equal_ch=True):
        super(MixConv2d, self).__init__()
        groups = len(k)
        if equal_ch:
            i = torch.linspace(0, groups - 1E-6, c2).floor()
            c_ = [(i == g).sum() for g in range(groups)]
        else:
            b = [c2] + [0] * groups
            a = np.eye(groups + 1, groups, k=-1)
            a -= np.roll(a, 1, axis=1)
            a *= np.array(k) ** 2
            a[0] = 1
            c_ = np.linalg.lstsq(a, b, rcond=None)[0].round()

        self.m = nn.ModuleList([nn.Conv2d(c1, int(c_[g]), k[g], s, k[g] // 2, bias=False) for g in range(groups)])
        self.bn = nn.BatchNorm2d(c2)
        self.act = nn.LeakyReLU(0.1, inplace=True)

    def forward(self, x):
        return x + self.act(self.bn(torch.cat([m(x) for m in self.m], 1)))


class Ensemble(nn.ModuleList):
    def __init__(self):
        super(Ensemble, self).__init__()

    def forward(self, x, augment=False):
        y = []
        for module in self:
            y.append(module(x, augment)[0])
        y = torch.cat(y, 1)
        return y, None





class ORT_NMS(torch.autograd.Function):
    '''ONNX-Runtime NMS operation'''
    @staticmethod
    def forward(ctx,
                boxes,
                scores,
                max_output_boxes_per_class=torch.tensor([100]),
                iou_threshold=torch.tensor([0.45]),
                score_threshold=torch.tensor([0.25])):
        device = boxes.device
        batch = scores.shape[0]
        num_det = random.randint(0, 100)
        batches = torch.randint(0, batch, (num_det,)).sort()[0].to(device)
        idxs = torch.arange(100, 100 + num_det).to(device)
        zeros = torch.zeros((num_det,), dtype=torch.int64).to(device)
        selected_indices = torch.cat([batches[None], zeros[None], idxs[None]], 0).T.contiguous()
        selected_indices = selected_indices.to(torch.int64)
        return selected_indices

    @staticmethod
    def symbolic(g, boxes, scores, max_output_boxes_per_class, iou_threshold, score_threshold):
        return g.op("NonMaxSuppression", boxes, scores, max_output_boxes_per_class, iou_threshold, score_threshold)


class TRT_NMS(torch.autograd.Function):
    '''TensorRT NMS operation'''
    @staticmethod
    def forward(
        ctx,
        boxes,
        scores,
        background_class=-1,
        box_coding=1,
        iou_threshold=0.45,
        max_output_boxes=100,
        plugin_version="1",
        score_activation=0,
        score_threshold=0.25,
    ):
        batch_size, num_boxes, num_classes = scores.shape
        num_det = torch.randint(0, max_output_boxes, (batch_size, 1), dtype=torch.int32)
        det_boxes = torch.randn(batch_size, max_output_boxes, 4)
        det_scores = torch.randn(batch_size, max_output_boxes)
        det_classes = torch.randint(0, num_classes, (batch_size, max_output_boxes), dtype=torch.int32)
        return num_det, det_boxes, det_scores, det_classes

    @staticmethod
    def symbolic(g,
                 boxes,
                 scores,
                 background_class=-1,
                 box_coding=1,
                 iou_threshold=0.45,
                 max_output_boxes=100,
                 plugin_version="1",
                 score_activation=0,
                 score_threshold=0.25):
        out = g.op("TRT::EfficientNMS_TRT",
                   boxes,
                   scores,
                   background_class_i=background_class,
                   box_coding_i=box_coding,
                   iou_threshold_f=iou_threshold,
                   max_output_boxes_i=max_output_boxes,
                   plugin_version_s=plugin_version,
                   score_activation_i=score_activation,
                   score_threshold_f=score_threshold,
                   outputs=4)
        nums, boxes, scores, classes = out
        return nums, boxes, scores, classes


class ONNX_ORT(nn.Module):
    '''onnx module with ONNX-Runtime NMS operation.'''
    def __init__(self, max_obj=100, iou_thres=0.45, score_thres=0.25, max_wh=640, device=None, n_classes=80):
        super().__init__()
        self.device = device if device else torch.device("cpu")
        self.max_obj = torch.tensor([max_obj]).to(device)
        self.iou_threshold = torch.tensor([iou_thres]).to(device)
        self.score_threshold = torch.tensor([score_thres]).to(device)
        self.max_wh = max_wh
        self.convert_matrix = torch.tensor([[1, 0, 1, 0], [0, 1, 0, 1], [-0.5, 0, 0.5, 0], [0, -0.5, 0, 0.5]],
                                           dtype=torch.float32,
                                           device=self.device)
        self.n_classes=n_classes

    def forward(self, x):
        boxes = x[:, :, :4]
        conf = x[:, :, 4:5]
        scores = x[:, :, 5:]
        if self.n_classes == 1:
            scores = conf
        else:
            scores *= conf
        boxes @= self.convert_matrix
        max_score, category_id = scores.max(2, keepdim=True)
        dis = category_id.float() * self.max_wh
        nmsbox = boxes + dis
        max_score_tp = max_score.transpose(1, 2).contiguous()
        selected_indices = ORT_NMS.apply(nmsbox, max_score_tp, self.max_obj, self.iou_threshold, self.score_threshold)
        X, Y = selected_indices[:, 0], selected_indices[:, 2]
        selected_boxes = boxes[X, Y, :]
        selected_categories = category_id[X, Y, :].float()
        selected_scores = max_score[X, Y, :]
        X = X.unsqueeze(1).float()
        return torch.cat([X, selected_boxes, selected_categories, selected_scores], 1)

class ONNX_TRT(nn.Module):
    def __init__(self, max_obj=100, iou_thres=0.45, score_thres=0.25, max_wh=None ,device=None, n_classes=80):
        super().__init__()
        assert max_wh is None
        self.device = device if device else torch.device('cpu')
        self.background_class = -1,
        self.box_coding = 1,
        self.iou_threshold = iou_thres
        self.max_obj = max_obj
        self.plugin_version = '1'
        self.score_activation = 0
        self.score_threshold = score_thres
        self.n_classes=n_classes

    def forward(self, x):
        boxes = x[:, :, :4]
        conf = x[:, :, 4:5]
        scores = x[:, :, 5:]
        if self.n_classes == 1:
            scores = conf
        else:
            scores *= conf
        num_det, det_boxes, det_scores, det_classes = TRT_NMS.apply(boxes, scores, self.background_class, self.box_coding,
                                                                    self.iou_threshold, self.max_obj,
                                                                    self.plugin_version, self.score_activation,
                                                                    self.score_threshold)
        return num_det, det_boxes, det_scores, det_classes


class End2End(nn.Module):
    def __init__(self, model, max_obj=100, iou_thres=0.45, score_thres=0.25, max_wh=None, device=None, n_classes=80):
        super().__init__()
        device = device if device else torch.device('cpu')
        assert isinstance(max_wh,(int)) or max_wh is None
        self.model = model.to(device)
        self.model.model[-1].end2end = True
        self.patch_model = ONNX_TRT if max_wh is None else ONNX_ORT
        self.end2end = self.patch_model(max_obj, iou_thres, score_thres, max_wh, device, n_classes)
        self.end2end.eval()

    def forward(self, x):
        x = self.model(x)
        x = self.end2end(x)
        return x






def attempt_load(weights, map_location=None, inplace=True, fuse=True):
    model = []
    for w in weights if isinstance(weights, list) else [weights]:
        attempt_download(w)
        ckpt = torch.load(w, map_location=map_location, weights_only=False)
        model.append(ckpt['ema' if ckpt.get('ema') else 'model'].float().fuse().eval())
    return model[0] if len(model) == 1 else model
    for m in model.modules():
        if type(m) in [nn.Hardswish, nn.LeakyReLU, nn.ReLU, nn.ReLU6, nn.SiLU]:
            m.inplace = True
        elif type(m) is nn.Upsample:
            m.recompute_scale_factor = None
        elif type(m) is Conv:
            m._non_persistent_buffers_set = set()

    if len(model) == 1:
        return model[-1]
    else:
        print('Ensemble created with %s\n' % weights)
        for k in ['names', 'stride']:
            setattr(model, k, getattr(model[-1], k))
        return model

In [ ]:
yolov7_path = "./yolov7.pt"
yolov7_tiny_path = "./yolov7-tiny.pt"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_v7 = attempt_load(yolov7_path, map_location=device)
model_tiny = attempt_load(yolov7_tiny_path, map_location=device)

model_v7.eval()
model_tiny.eval()

In [ ]:
def draw_boxes(image, detections, label=""):
    for *xyxy, conf, cls in detections:
        x1, y1, x2, y2 = map(int, xyxy)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"{label} {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return image


In [ ]:

def yolov7onframe_onlyPerson(frame):
    ll = "-1"
    rxyxy = "-1"

    try:
      img = letterbox(frame, new_shape=640)[0]
      img = img[:, :, ::-1].transpose(2, 0, 1)
      img = np.ascontiguousarray(img)

      img = torch.from_numpy(img).to(device)
      img = img.float()
      img /= 255.0
      if img.ndimension() == 3:
          img = img.unsqueeze(0)

      with torch.no_grad():
          pred = model_tiny(img)[0]
      pred = non_max_suppression(pred, 0.5, 0.45, classes=None, agnostic=False)

      for i, det in enumerate(pred):
          if len(det):
              det[:, :4] = scale_coords(img.shape[2:], det[:, :4], frame.shape).round()

              for *xyxy, conf, cls in reversed(det):
                  label = f'{model.names[int(cls)]} {conf:.2f}'
                  if("person" in label):
                    print(label)
                    ll = label
                    rxyxy = xyxy
                    print(*xyxy)
    except:
        pass


    return ll,frame,rxyxy


In [ ]:
import numpy as np
from scipy.fft import fft, fftfreq

def dominant_frequency(signal, fps=25):
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1 / fps)[:N // 2]

    amplitudes = np.abs(yf[:N // 2])
    dominant_freq = xf[np.argmax(amplitudes)]

    return dominant_freq

In [ ]:
def convert_xyxy_to_bbox(xyxy):
    x1, y1, x2, y2 = [int(val.item()) for val in xyxy]
    return (x1, y1, x2 - x1, y2 - y1)

In [ ]:
indexes = []
is_coord = []
coordinatess = []

In [ ]:
cap = cv2.VideoCapture("/content/drive/MyDrive/baba_legzes_2.mp4")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print( length )

In [ ]:
frames = []
for i in range(1500):
    ret, frame = cap.read()
    if(ret==True):
      frames.append(frame)

In [ ]:
import time
import numpy as np

def average_vector(flow):
    return [sum(sum(flow))/len(flow)][0]


def calc_rep(flow):
    magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    return np.mean(magnitude)


def calcLenght(flow):
  return sum(sum(flow[:,:,0]*flow[:,:,0]))+sum(sum(flow[:,:,1]*flow[:,:,1]))

In [ ]:
%matplotlib inline
cap = cv2.VideoCapture("/content/drive/MyDrive/baba_legzes_2.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"FPS: {fps}")
cap.release()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

def calc_rep(flow):
    magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    return np.mean(magnitude)

blur_kernel_size = (5, 5)
motion_amplitudes = []

frame1 = frames[0][300:900, 300:1000]
frame1 = cv2.resize(frame1, (500, 500))
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
prvs = cv2.GaussianBlur(prvs, blur_kernel_size, 0)

for i in range(1, len(frames)):
    frame = frames[i][300:900, 300:1000]
    frame = cv2.resize(frame, (500, 500))
    next_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    next_frame = cv2.GaussianBlur(next_frame, blur_kernel_size, 0)

    flow = cv2.calcOpticalFlowFarneback(prvs, next_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    motion_amplitudes.append(calc_rep(flow))

    prvs = next_frame

print(" Mozgásdetekció kész. Értékek száma:", len(motion_amplitudes))

plt.figure(figsize=(12, 5))
plt.plot(motion_amplitudes)
plt.title("Mozgási amplitúdó görbe")
plt.xlabel("Képkocka index")
plt.ylabel("Mozgás mértéke")
plt.grid(True)
plt.show()


In [ ]:

!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

import clip
import torch
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)


In [ ]:

import time
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt

def run_yolo(model, image):
    img = letterbox(image, new_shape=640)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontiguousarray(img)
    img_tensor = torch.from_numpy(img).to(device).float() / 255.0
    if img_tensor.ndimension() == 3:
        img_tensor = img_tensor.unsqueeze(0)

    with torch.no_grad():
        start = time.time()
        pred = model(img_tensor)[0]
        inference_time = time.time() - start

        pred = non_max_suppression(pred, 0.05, 0.45)
        det = pred[0]

        if det is not None and len(det):
            det[:, :4] = scale_coords(img_tensor.shape[2:], det[:, :4], image.shape).round()
        return det.cpu().numpy() if det is not None else [], inference_time


def draw_boxes(image, detections, label="Model"):
    for *xyxy, conf, cls in detections:
        x1, y1, x2, y2 = map(int, xyxy)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"{label} {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 1)
    return image

cap = cv2.VideoCapture("/content/drive/MyDrive/baba_legzes_2.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
output_size = (width * 2, height)
out_video = cv2.VideoWriter("yolo_baba_1_hosszu.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, output_size)

frame_idx = 0
frame_limit = 100

while cap.isOpened() and frame_idx < frame_limit:
    ret, frame = cap.read()
    if not ret:
        break

    print(f"\n Frame {frame_idx}:")

    det_v7, _ = run_yolo(model_v7, frame)
    det_tiny, _ = run_yolo(model_tiny, frame)

    conf_v7 = np.mean([d[4] for d in det_v7]) if len(det_v7) else 0
    conf_tiny = np.mean([d[4] for d in det_tiny]) if len(det_tiny) else 0

    print(f"YOLOv7 - Conf: {conf_v7:.2f}, Detections: {len(det_v7)}")
    print(f"YOLOv7-Tiny - Conf: {conf_tiny:.2f}, Detections: {len(det_tiny)}")

    best_v7 = [max(det_v7, key=lambda d: d[4])] if len(det_v7) else []
    img_v7 = draw_boxes(frame.copy(), best_v7, "YOLOv7")

    best_tiny = [max(det_tiny, key=lambda d: d[4])] if len(det_tiny) else []
    img_tiny = draw_boxes(frame.copy(), best_tiny, "YOLOv7-Tiny")

    combined = np.hstack((img_v7, img_tiny))

    out_video.write(combined)

    frame_idx += 1


cap.release()
out_video.release()
print(" Kimeneti videó elkészült")

det_v7, time_v7 = run_yolo(model_v7, frame)
gpu_v7 = torch.cuda.memory_allocated() / 1e6 if torch.cuda.is_available() else 0
conf_v7 = np.mean([d[4] for d in det_v7]) if len(det_v7) else 0

det_tiny, time_tiny = run_yolo(model_tiny, frame)
gpu_tiny = torch.cuda.memory_allocated() / 1e6 if torch.cuda.is_available() else 0
conf_tiny = np.mean([d[4] for d in det_tiny]) if len(det_tiny) else 0

print(f"YOLOv7:")
print(f" - Inference time: {time_v7:.4f}s")
print(f" - Detections: {len(det_v7)}")
print(f" - Avg confidence: {conf_v7:.2f}")
print(f" - GPU memory used: {gpu_v7:.2f} MB\n")

print(f"YOLOv7-Tiny:")
print(f" - Inference time: {time_tiny:.4f}s")
print(f" - Detections: {len(det_tiny)}")
print(f" - Avg confidence: {conf_tiny:.2f}")
print(f" - GPU memory used: {gpu_tiny:.2f} MB\n")

best_v7 = [max(det_v7, key=lambda d: d[4])] if len(det_v7) else []
best_tiny = [max(det_tiny, key=lambda d: d[4])] if len(det_tiny) else []

img_v7 = draw_boxes(frame.copy(), best_v7, "YOLOv7")
img_tiny = draw_boxes(frame.copy(), best_tiny, "YOLOv7-Tiny")

plt.figure(figsize=(16, 7))
plt.subplot(1, 2, 1)
plt.title("YOLOv7")
plt.imshow(cv2.cvtColor(img_v7, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("YOLOv7-Tiny")
plt.imshow(cv2.cvtColor(img_tiny, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.show()



In [ ]:
import clip
import torch
from PIL import Image
import cv2
from datetime import timedelta

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

clip_prompts = [
    "a baby alone in an incubator",
    "a baby with a nurse's hand nearby",
    "a baby being touched by a hand",
    "a baby lying without anyone nearby"
]
clip_texts = clip.tokenize(clip_prompts).to(device)

def clip_detect_hand_near_baby(cropped_baby_frame, threshold=0.3):
    image = preprocess(Image.fromarray(cropped_baby_frame)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = clip_model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        text_features = clip_model.encode_text(clip_texts)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        similarity = (image_features @ text_features.T).squeeze(0)
        best_match_idx = similarity.argmax().item()
        best_score = similarity[best_match_idx].item()

        return "hand" in clip_prompts[best_match_idx] and best_score > threshold

def group_time_ranges(frame_statuses, fps):
    intervals = []
    if not frame_statuses:
        return intervals

    current_status = frame_statuses[0][1]
    start_frame = frame_statuses[0][0]

    for i in range(1, len(frame_statuses)):
        frame_idx, status = frame_statuses[i]
        if status != current_status:
            intervals.append((start_frame, frame_statuses[i - 1][0], current_status))
            start_frame = frame_idx
            current_status = status
    intervals.append((start_frame, frame_statuses[-1][0], current_status))
    return intervals

def format_time(seconds):
    return str(timedelta(seconds=int(seconds)))

cap = cv2.VideoCapture("/content/drive/MyDrive/baba_legzes_2.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
frame_idx = 0
frame_statuses = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detections, _ = run_yolo(model_v7, frame)
    baby_class = 0
    baby_found = False
    status = "nincs_baba"

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        if int(cls) == baby_class:
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            baby_crop = frame[y1:y2, x1:x2]
            if baby_crop.size == 0:
                continue

            baby_found = True
            has_hand = clip_detect_hand_near_baby(baby_crop)

            status = "kéz" if has_hand else "tiszta"
            break

    if not baby_found:
        status = "nincs_baba"

    frame_statuses.append((frame_idx, status))
    frame_idx += 1

cap.release()

print("\n Kéz jelenlétének időszakaszai:")

if frame_statuses:
    prev_status = frame_statuses[0][1]
    start_f = frame_statuses[0][0]

    for i in range(1, len(frame_statuses)):
        frame_idx, status = frame_statuses[i]
        if status != prev_status:
            start_time = str(timedelta(seconds=int(start_f // fps)))
            end_time = str(timedelta(seconds=int(frame_statuses[i - 1][0] // fps)))

            if prev_status == "kéz":
                print(f" Kéz a képen: {start_time} - {end_time}")
            elif prev_status == "tiszta":
                print(f" Tiszta időszak: {start_time} - {end_time}")
            else:
                print(f" Baba nem látszik: {start_time} - {end_time}")

            start_f = frame_idx
            prev_status = status

    # utolsó szakasz kiírása
    start_time = str(timedelta(seconds=int(start_f // fps)))
    end_time = str(timedelta(seconds=int(frame_statuses[-1][0] // fps)))
    if prev_status == "kéz":
        print(f" Kéz a képen: {start_time} - {end_time}")
    elif prev_status == "tiszta":
        print(f" Tiszta időszak: {start_time} - {end_time}")
    else:
        print(f" Baba nem látszik: {start_time} - {end_time}")


In [ ]:
import matplotlib.pyplot as plt

additional_prompts = [
    "a hand reaching into the incubator",
    "a nurse touching the baby",
    "a human hand near an infant",
    "a caregiver hand over the baby",
    "a person placing hand on baby",
    "a nurse placing hand close to a baby"
]

kezmentes_intervallumok = [(0, 133), (262, 267), (332, 575)]
cap = cv2.VideoCapture("/content/drive/MyDrive/baba_legzes_2.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()

def is_frame_clean(frame_idx):
    sec = frame_idx / fps
    return any(start <= sec <= end for start, end in kezmentes_intervallumok)

prompt_counts = []
false_positives = []

for i in range(1, len(additional_prompts) + 1):
    print(f"\n {i+4} prompttal történő detekció...")

    all_prompts = clip_prompts + additional_prompts[:i]
    tokenized = clip.tokenize(all_prompts).to(device)

    fp_count = 0
    checked_frames = 0

    cap = cv2.VideoCapture("/content/drive/MyDrive/baba_legzes_2.mp4")
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if not is_frame_clean(frame_idx):
            frame_idx += 1
            continue

        detections, _ = run_yolo(model_v7, frame)
        for det in detections:
            x1, y1, x2, y2, conf, cls = det
            if int(cls) == 0:
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                baby_crop = frame[y1:y2, x1:x2]
                if baby_crop.size == 0:
                    continue

                image = preprocess(Image.fromarray(baby_crop)).unsqueeze(0).to(device)
                with torch.no_grad():
                    image_features = clip_model.encode_image(image)
                    image_features /= image_features.norm(dim=-1, keepdim=True)
                    text_features = clip_model.encode_text(tokenized)
                    text_features /= text_features.norm(dim=-1, keepdim=True)
                    similarity = (image_features @ text_features.T).squeeze(0)
                    best_match_idx = similarity.argmax().item()
                    best_prompt = all_prompts[best_match_idx]

                    if "hand" in best_prompt or "nurse" in best_prompt:
                        fp_count += 1
                break
        frame_idx += 1
    cap.release()

    prompt_counts.append(i + 4)
    false_positives.append(fp_count)

plt.figure(figsize=(10, 5))
plt.plot(prompt_counts, false_positives, marker='o')
plt.title("False positive detekciók száma a promptok számának függvényében")
plt.xlabel("CLIP promptok száma")
plt.ylabel("Téves 'kéz' detekciók (kézmentes szakaszokon)")
plt.grid(True)
plt.show()


In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture("/content/drive/MyDrive/baba_legzes_2.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)

crop_size = 320
out = cv2.VideoWriter("tiszta_baba_cropok_fixkeret_baba_feje_nem_latszik.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (crop_size, crop_size))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detections, _ = run_yolo(model_v7, frame)
    baby_class = 0

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        if int(cls) == baby_class:
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            baby_crop = frame[y1:y2, x1:x2]
            if baby_crop.size == 0:
                continue

            has_hand = clip_detect_hand_near_baby(baby_crop)

            if not has_hand:
                cx = (x1 + x2) // 2
                cy = (y1 + y2) // 2

                half = crop_size // 2
                x1_crop = max(0, cx - half)
                y1_crop = max(0, cy - half)
                x2_crop = x1_crop + crop_size
                y2_crop = y1_crop + crop_size

                if x2_crop > frame.shape[1]:
                    x1_crop = frame.shape[1] - crop_size
                    x2_crop = frame.shape[1]
                if y2_crop > frame.shape[0]:
                    y1_crop = frame.shape[0] - crop_size
                    y2_crop = frame.shape[0]

                fixed_crop = frame[y1_crop:y2_crop, x1_crop:x2_crop]

                if fixed_crop.shape[:2] == (crop_size, crop_size):
                    out.write(fixed_crop)

            break

cap.release()
out.release()
print(" Elkészült a 'tiszta' baba cropokat tartalmazó videó: tiszta_baba_cropok_fixkeret.mp4")


In [ ]:
cap = cv2.VideoCapture("/content/drive/MyDrive/baba_legzes_2.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("tiszta_baba_cropok2_baba_1.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (224, 224))
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detections, _ = run_yolo(model_v7, frame)
    baby_class = 0
    baby_found = False

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        if int(cls) == baby_class:
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            baby_crop = frame[y1:y2, x1:x2]
            if baby_crop.size == 0:
                continue

            baby_found = True
            has_hand = clip_detect_hand_near_baby(baby_crop)

            if not has_hand:
                resized_crop = cv2.resize(baby_crop, (224, 224))
                out.write(resized_crop)

            break
    frame_idx += 1

cap.release()
out.release()
print(" Elkészült a 'tiszta' baba cropokat tartalmazó videó: tiszta_baba_cropok.mp4")
